In [8]:
# Libraries
import numpy as np
import pandas as pd
import re
from datetime import datetime
import requests
from bs4 import BeautifulSoup # pip install beautifulsoup4
import pandas as pd
import requests
import re

In [9]:
df_ta = pd.read_csv(r'C:\Users\Tim_K\Ironhack\Project2\data\TA_restaurants_curated.csv')

In [10]:
valid_city = ['Barcelona']
df_ta.loc[~df_ta['City'].isin(valid_city), 'City'] = np.nan

df_ta = df_ta.dropna(subset=['City'])
# Now we have 8425 restaurants left, only in Barcelona

# Dropping all columns we don't need
df_ta.drop(columns='Ranking',inplace=True)
df_ta.drop(columns='Reviews',inplace=True)
df_ta.drop(columns='ID_TA',inplace=True)
df_ta.drop(columns='URL_TA',inplace=True)
df_ta.drop(columns='Price Range',inplace=True)
df_ta.drop(columns='City',inplace=True)
df_ta.drop(columns='Unnamed: 0',inplace=True)

def rating_to_percentage(rating):
    if pd.isnull(rating):
        return rating
    return int(rating * 20)

df_ta['Rating'] = df_ta['Rating'].apply(rating_to_percentage)

df_ta['Rating'] = df_ta['Rating'].apply(lambda x: str(int(x)) + '%' if not pd.isna(x) else x)
df_ta['Number of Reviews'] = df_ta['Number of Reviews'].apply(lambda x: str(int(x)) if not pd.isna(x) else x)
df_ta

,Name,Cuisine Style,Rating,Number of Reviews
5372,Uma,"['International', 'Mediterranean', 'Fusion', '...",100%,792
5373,Viana,"['International', 'Mediterranean', 'Spanish', ...",100%,2707
5374,Blavis,"['Mediterranean', 'European', 'Spanish', 'Vege...",100%,643
5375,My Restaurant,"['Mediterranean', 'European', 'Spanish', 'Vege...",100%,159
5376,Bodega Biarritz,"['Mediterranean', 'Spanish', 'Wine Bar', 'Euro...",100%,1078
...,...,...,...,...
13792,Aqui hi ha molla Gastrobar,"['Mediterranean', 'Spanish', 'Fusion', 'Diner']",NaN,NaN
13793,Istanbul Grill Donor Kebab,['Turkish'],NaN,NaN
13794,Natsu Sushi,['Japanese'],NaN,NaN
13795,Zi Zhu Lin,['Chinese'],NaN,NaN


In [11]:
def scrape_glovo_restaurants(url_template, pages):
    all_data = []

    for page_num in range(1, pages+1):
        url = url_template.format(page_num)
        html = requests.get(url)
        soup = BeautifulSoup(html.content, "html.parser")
# First we get the name of the restaurant
        tags_name = soup.find_all('h3', attrs = {"class":"card-title"})
        names = [i.getText().strip() for i in tags_name]
# Then the type of cuisine the restaurant serves, this can be more than 1 style
        cuisine_style = soup.find_all('div', attrs = {"class":"store-filter-container"})
        cuisine_style = [i.getText().replace("\n", "").strip().split('                      ') for i in cuisine_style]
# Then the rating (from 0-100%, 100% being the highest rating)
        rating = soup.find_all('span', attrs = {"class":"store-card-rating-info__label"})
        rating = [i.getText().strip() for i in rating]
# Amount of reviews, 500+ being the highest amount
        number_reviews = soup.find_all('span', attrs = {"class":"store-card-rating-info__label--secondary"})
        number_reviews = [re.search(r'\((.*?)\)', i.getText().strip()).group(1) for i in number_reviews]

# Adding all information to a dataframe
        for name, style, rating, num_reviews in zip(names, cuisine_style, rating, number_reviews):
            data = {
                'Name': name,
                'Cuisine Style': style,
                'Rating Glovo': rating,
                'Number of Reviews': num_reviews
            }
            all_data.append(data)

    df = pd.DataFrame(all_data)
    return df


In [12]:
url_template = "https://glovoapp.com/es/en/barcelona/restaurants_1/?page={}"

df_glovo = scrape_glovo_restaurants(url_template, 79)

In [18]:
merged_df = pd.merge(df_ta, df_glovo, on='Name', how='inner')

merged_df = merged_df.rename(columns={
    'Rating': 'Rating TA',
    'Number of Reviews_x': 'Number of Reviews TA',
    'Cuisine Style_x': 'Cuisine Style TA',
    'Rating Glovo': 'Rating Glovo',
    'Number of Reviews_y': 'Number of Reviews Glovo',
    'Cuisine Style_y': 'Cuisine Style Glovo'
})
merged_df = merged_df[['Name', 'Rating TA', 'Rating Glovo', 'Cuisine Style TA', 'Cuisine Style Glovo', 'Number of Reviews TA', 'Number of Reviews Glovo']]
merged_df

,Name,Rating TA,Rating Glovo,Cuisine Style TA,Cuisine Style Glovo,Number of Reviews TA,Number of Reviews Glovo
0,Pizza Circus,90%,96%,"['Italian', 'Pizza', 'Fast Food', 'Vegetarian ...",[Pizza],517,500+
1,Pizza Circus,100%,96%,"['Italian', 'Pizza']",[Pizza],4,500+
2,A Tu Bola,90%,99%,"['Fast Food', 'Mediterranean', 'International'...",[International],1068,64
3,Macchina Pasta Bar,90%,95%,"['Italian', 'Mediterranean', 'Vegetarian Frien...",[Italian],1050,500+
4,Macchina Pasta Bar,90%,95%,"['Italian', 'Mediterranean', 'Vegetarian Frien...",[Italian],518,500+
...,...,...,...,...,...,...,...
256,Telepizza,20%,94%,NaN,[Pizza],NaN,317
257,Pita House,30%,92%,['Spanish'],"[Arabic, Pizza]",465,121
258,Vapiano,-20%,93%,"['Pizza', 'Mediterranean', 'Italian']","[Italian, Pizza]",2,391
259,Hong Kong,NaN,91%,['Chinese'],[Asian],NaN,316


In [19]:
#Now there are a few columns that have duplicat names, which are chains (McDonalds, Burger king, Pizza circus etc) we only keep 1 of them.
merged_df = merged_df.drop_duplicates(subset='Name', keep='first')
merged_df

,Name,Rating TA,Rating Glovo,Cuisine Style TA,Cuisine Style Glovo,Number of Reviews TA,Number of Reviews Glovo
0,Pizza Circus,90%,96%,"['Italian', 'Pizza', 'Fast Food', 'Vegetarian ...",[Pizza],517,500+
2,A Tu Bola,90%,99%,"['Fast Food', 'Mediterranean', 'International'...",[International],1068,64
3,Macchina Pasta Bar,90%,95%,"['Italian', 'Mediterranean', 'Vegetarian Frien...",[Italian],1050,500+
7,Palosanto,90%,--,"['Bar', 'Mediterranean', 'European', 'Spanish'...",[Mediterranean],953,16
8,Eat Caribbean,90%,93%,"['African', 'Caribbean', 'Fusion', 'Jamaican',...","[American, Latin American]",164,10
...,...,...,...,...,...,...,...
253,Telepizza,50%,94%,['Pizza'],[Pizza],14,317
257,Pita House,30%,92%,['Spanish'],"[Arabic, Pizza]",465,121
258,Vapiano,-20%,93%,"['Pizza', 'Mediterranean', 'Italian']","[Italian, Pizza]",2,391
259,Hong Kong,NaN,91%,['Chinese'],[Asian],NaN,316
